# General Info

<b>Due date</b>: Sunday, 5 Apr 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be working with a collection tweets. The task is to classify whether a tweet constitutes a rumour event. This homework involves writing code to preprocess data and perform text classification.

# 1. Preprocessing (5 marks)

**Instructions**: Run the code below to download the tweet corpus for the assignment. Note: the download may take some time. **No implementation is needed.**

In [1]:
import requests
import os
from pathlib import Path

fname = 'rumour-data.tgz'
data_dir = os.path.splitext(fname)[0] #'rumour-data'

my_file = Path(fname)
if not my_file.is_file():
    url = "https://github.com/jhlau/jhlau.github.io/blob/master/files/rumour-data.tgz?raw=true"
    r = requests.get(url)

    #Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)
        
print("Done. File downloaded:", my_file)


Done. File downloaded: rumour-data.tgz


**Instructions**: Run the code to extract the zip file. Note: the extraction may take a minute or two. **No implementation is needed.**

In [2]:
import tarfile

#decompress rumour-data.tgz
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

#remove superfluous files (e.g. .DS_store)
extra_files = []
for r, d, f in os.walk(data_dir):
    for file in f:
        if (file.startswith(".")):
            extra_files.append(os.path.join(r, file))
for f in extra_files:
    os.remove(f)

print("Extraction done.")

Extraction done.


### Question 1 (1.0 mark)

**Instructions**: The corpus data is in the *rumour-data* folder. It contains 2 sub-folders: *non-rumours* and *rumours*. As the names suggest, *rumours* contains all rumour-propagating tweets, while *non-rumours* has normal tweets. Within  *rumours* and *non-rumours*, you'll find some sub-folders, each named with an ID. Each of these IDs constitutes an 'event', where an event is defined as consisting a **source tweet** and its **reactions**.

An illustration of the folder structure is given below:

    rumour-data
        - rumours
            - 498254340310966273
                - reactions
                    - 498254340310966273.json
                    - 498260814487642112.json
                - source-tweet
                    - 498254340310966273.json
        - non-rumours

Now we need to gather the tweet messages for rumours and non-rumour events. As the individual tweets are stored in json format, we need to use a json parser to parse and collect the actual tweet message. The function `get_tweet_text_from_json(file_path)` is provided to do that.

**Task**: Complete the `get_events(event_dir)` function. The function should return **a list of events** for a particular class of tweets (e.g. rumours), and each event should contain the source tweet message and all reaction tweet messages.

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [3]:
import json

def get_tweet_text_from_json(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
        return data["text"]
    
import os
def get_filenames(path):
    files = []
    # r = root, d = directories, f = files
    for r, d, f in os.walk(path+"/source-tweet"):
        for file in f:
            if '.json' in file:
                files.append(os.path.join(r, file))
    for r, d, f in os.walk(path+"/reactions"):
        for file in f:
            if '.json' in file:
                files.append(os.path.join(r, file))
    return files
    
def get_events(event_dir):
    event_list = []
    for event in sorted(os.listdir(event_dir)):
        file_names = get_filenames(os.path.join(os.getcwd(), event_dir, event)) # Current path + event category + event...
        current_event = []
        for each in file_names:
            current_event.append(get_tweet_text_from_json(each))
        event_list.append(current_event)
    
    return event_list
    
#a list of events, and each event is a list of tweets (source tweet + reactions)    
rumour_events = get_events(os.path.join(data_dir, "rumours"))
nonrumour_events = get_events(os.path.join(data_dir, "non-rumours"))

print("Number of rumour events =", len(rumour_events))
print("Number of non-rumour events =", len(nonrumour_events))

Number of rumour events = 500
Number of non-rumour events = 1000


**For your testing:**

In [4]:
assert(len(rumour_events) == 500)
assert(len(nonrumour_events) == 1000)

### Question 2 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation. The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); and (2) remove stopwords (based on NLTK `stopwords`).

**Task**: Complete the `preprocess_events(event)` function. The function takes **a list of events** as input, and returns **a list of preprocessed events**. Each preprocessed event should have a dictionary of words and frequencies.

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [5]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter

tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))

def preprocess_events(events):
    dict_list = []
    for event in events:
        tokens = []
        for tweet in event:
            tweetTokens = tt.tokenize(tweet)
            tweetTokens = [word for word in tweetTokens if word.lower() not in stopwords]
            tokens.append(tweetTokens)
        eventTokens = [item for sublist in tokens for item in sublist]
        dictionary = dict(Counter(eventTokens))
        dict_list.append(dictionary)
    return dict_list        

preprocessed_rumour_events = preprocess_events(rumour_events)
preprocessed_nonrumour_events = preprocess_events(nonrumour_events)

print("Number of preprocessed rumour events =", len(preprocessed_rumour_events))
print("Number of preprocessed non-rumour events =", len(preprocessed_nonrumour_events))

Number of preprocessed rumour events = 500
Number of preprocessed non-rumour events = 1000


**For your testing**:

In [6]:
assert(len(preprocessed_rumour_events) == 500)
assert(len(preprocessed_nonrumour_events) == 1000)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [7]:
def get_all_hashtags(events):
    hashtags = set([])
    for event in events:
        for word, frequency in event.items():
            if word.startswith("#"):
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(preprocessed_rumour_events + preprocessed_nonrumour_events)
print("Number of hashtags =", len(hashtags))

Number of hashtags = 1829


### Question 3 (2.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing a reversed version of the MaxMatch algorithm discussed in class, where matching begins at the end of the hashtag and progresses backwards. NLTK has a list of words that you can use for matching, see starter code below. Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching. When lemmatising a word, you also need to provide the part-of-speech tag of the word. You should use `nltk.tag.pos_tag` for doing part-of-speech tagging.

Note that the list of words is incomplete, and, if you are unable to make any longer match, your code should default to matching a single letter. Create a new list of tokenized hashtags (this should be a list of lists of strings) and use slicing to print out the last 20 hashtags in the list.

For example, given "#speakup", the algorithm should produce: \["#", "speak", "up"\]. And note that you do not need to delete the hashtag symbol ("#") from the tokenised outputs.

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing a reversed MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of word tokens".

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [8]:
from nltk.corpus import wordnet

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK

def word_postag(word):
    tag = nltk.tag.pos_tag([word])[0][1][0]
    tag_set = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_set.get(tag, wordnet.NOUN)

def reverse_max_match(hashtag):
    if not hashtag:
        return []
    max_match = ""
    location = 0
    for i in range(0, len(hashtag)):
        first_word = (hashtag[-i:]) # Get last i characters...
        tag = word_postag(first_word)
        first_lemma = lemmatizer.lemmatize(first_word, pos = tag)
        remainder = hashtag[0:len(hashtag) - i] # Get first (length - i) characters...
        if first_word in words and len(first_word) > len(max_match):
            max_match = first_word
            location = i
    
    if len(max_match) > 0: # Match found!
        remainder = hashtag[0:len(hashtag) - location]
        return reverse_max_match(remainder) + [max_match]
        

    # No match found; make one-letter word...
    first_word = hashtag[len(hashtag) - 1]
    remainder = hashtag[0:len(hashtag) - 1]

    return reverse_max_match(remainder) + [first_word]

def tokenize_hashtags(hashtags):
    hashtag_tokens = {}
    for hashtag in hashtags:
        tokens = reverse_max_match(hashtag)
        hashtag_tokens[hashtag] = tokens
    
    return hashtag_tokens

tokenized_hashtags = tokenize_hashtags(hashtags)

print(list(tokenized_hashtags.items())[:20])

[('#Sickular', ['#', 'S', 'i', 'c', 'k', 'u', 'lar']), ('#Patriots', ['#', 'Pat', 'riot', 's']), ('#Chemtrails', ['#', 'C', 'hem', 'trail', 's']), ('#lies', ['#', 'li', 'es']), ('#AlpsCrash', ['#', 'Al', 'p', 's', 'C', 'rash']), ('#SydenySiege', ['#', 'S', 'y', 'deny', 'S', 'ie', 'ge']), ('#Sotlof', ['#', 'S', 'o', 't', 'lof']), ('#TeamHumanity', ['#', 'T', 'e', 'am', 'Huma', 'nit', 'y']), ('#stlouis', ['#', 'st', 'lo', 'u', 'is']), ('#TruthNorthStrong', ['#', 'T', 'ruth', 'N', 'or', 'th', 'S', 'tron', 'g']), ('#Waco', ['#', 'Waco']), ('#Germanwing', ['#', 'German', 'wing']), ('#Islamic_State', ['#', 'Islamic', '_', 'State']), ('#MoreMilk', ['#', 'M', 'ore', 'M', 'ilk']), ('#proud', ['#', 'p', 'roud']), ('#pigs', ['#', 'pig', 's']), ('#weak', ['#', 'weak']), ('#EzellFord', ['#', 'E', 'z', 'ell', 'For', 'd']), ('#youidjit', ['#', 'you', 'id', 'j', 'it']), ('#Kafirphobic', ['#', 'Kafir', 'phobic'])]


**For your testing:**

In [9]:
assert(len(tokenized_hashtags) == len(hashtags))

### Question 4 (1.0 mark)

**Instructions**: Now that we have the tokenized hashtags, we need to go back and update the bag-of-words representation for each event.

**Task**: Complete the ``update_event_bow(events)`` function. The function takes **a list of preprocessed events**, and for each event, it looks for every hashtag it has and updates the bag-of-words dictionary with the tokenized hashtag tokens. Note: you do not need to delete the counts of the original hashtags when updating the bag-of-words (e.g., if a document has "#speakup":2 in its bag-of-words representation, you do not need to delete this hashtag and its counts).

In [10]:
def update_event_bow(events):
    for event in events:
        hashtags = [key for key, value in event.items() if key.startswith("#")] # Get all hashtags from every dictionary...
        for hashtag in hashtags:
            tokens = tokenized_hashtags[hashtag]
            dictionary = dict(Counter(tokens))
            del dictionary['#']
            event.update(dictionary)
            
update_event_bow(preprocessed_rumour_events)
update_event_bow(preprocessed_nonrumour_events)

print("Number of preprocessed rumour events =", len(preprocessed_rumour_events))
print("Number of preprocessed non-rumour events =", len(preprocessed_nonrumour_events))

Number of preprocessed rumour events = 500
Number of preprocessed non-rumour events = 1000


# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict, given a tweet and its reactions, whether it is a rumour or not. The task here is to create training, development and test partitions from the preprocessed events and convert the bag-of-words representation into feature vectors.

**Task**: Using scikit-learn, create training, development and test partitions with a 60%/20%/20% ratio. Remember to preserve the ratio of rumour/non-rumour events for all your partitions. Next, turn the bag-of-words dictionary of each event into a feature vector, using scikit-learn `DictVectorizer`.

In [11]:
from sklearn.feature_extraction import DictVectorizer
import numpy as np
from sklearn.model_selection import train_test_split

vectorizer = DictVectorizer(sparse=False)

# Truth labels
y_rumour = np.zeros(500)
y_non_rumour = np.ones(1000)

y_rumour = y_rumour.astype(np.uint8)
y_non_rumour = y_non_rumour.astype(np.uint8)

# Split non-rumour data...
X_train_nonrumour, X_test_nonrumour, y_train_nonrumour, y_test_nonrumour = train_test_split(preprocessed_nonrumour_events, y_non_rumour, test_size=0.2,shuffle = True, random_state=1)
X_train_nonrumour, X_dev_nonrumour, y_train_nonrumour, y_dev_nonrumour = train_test_split(X_train_nonrumour, y_train_nonrumour, test_size=0.25,shuffle = True, random_state=1)

# Split rumour data...
X_train_rumour, X_test_rumour, y_train_rumour, y_test_rumour = train_test_split(preprocessed_rumour_events, y_rumour, test_size=0.2,shuffle = True, random_state=1)
X_train_rumour, X_dev_rumour, y_train_rumour, y_dev_rumour = train_test_split(X_train_rumour, y_train_rumour, test_size=0.25,shuffle = True, random_state=1)

# Consolidate events...
X_train = X_train_nonrumour + X_train_rumour
X_dev = X_dev_nonrumour + X_dev_rumour
X_test = X_test_nonrumour + X_test_rumour

# Concatenate the result datasets...
y_train = np.concatenate((y_train_nonrumour, y_train_rumour))
y_dev = np.concatenate((y_dev_nonrumour, y_dev_rumour))
y_test = np.concatenate((y_test_nonrumour, y_test_rumour))

# Vectorise event dictionaries...
X_train_vect = vectorizer.fit_transform(X_train)
X_dev_vect = vectorizer.transform(X_dev)
X_test_vect = vectorizer.transform(X_test)

print("Vocabulary size =", len(vectorizer.vocabulary_))

Vocabulary size = 27931


### Question 6 (2.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation (hyper)parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the accuracy with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance for different hyper-parameter settings.

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Naive Bayes Multinomial NB's hyperparameter is alpha value of the model...
fit_options = ["True", "False"]
for choice in fit_options:
    for each in np.arange(0.1, 3.0, 0.1):
        clf_multinomial = MultinomialNB(alpha = each, fit_prior = choice)
        clf_multinomial.fit(X_train_vect, y_train)
        y_pred = clf_multinomial.predict(X_dev_vect)
        print("Accuracy for Alpha-value: " + str(round(each, 3)) + " with fit_prior " + choice + " is "+ str(round(accuracy_score(y_dev, y_pred), 3)) + ".")
    print("--------------------------")

# Linear Logistic Regression's hyperparameters are the solver type & C value (inverse of regularisation strength!)
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

# Avoiding 'lbfgs' becuase of the convergance warnings and ‘sag’, ‘saga’ because of our data (small; 900 records!)
solvers = ["newton-cg", "liblinear"] 

for s_type in solvers:
    for each in C:
        clf_regression = LogisticRegression(random_state=0, solver = s_type, C=each)
        clf_regression.fit(X_train_vect, y_train)
        y_pred = clf_regression.predict(X_dev_vect)
        print("Accuracy for C-value: "+ str(round(each, 3)) + " with solver type " + s_type + " is "+ str(round(accuracy_score(y_dev, y_pred), 3)) + ".")
    print("--------------------------")

Accuracy for Alpha-value: 0.1 with fit_prior True is 0.83.
Accuracy for Alpha-value: 0.2 with fit_prior True is 0.82.
Accuracy for Alpha-value: 0.3 with fit_prior True is 0.813.
Accuracy for Alpha-value: 0.4 with fit_prior True is 0.81.
Accuracy for Alpha-value: 0.5 with fit_prior True is 0.817.
Accuracy for Alpha-value: 0.6 with fit_prior True is 0.817.
Accuracy for Alpha-value: 0.7 with fit_prior True is 0.82.
Accuracy for Alpha-value: 0.8 with fit_prior True is 0.82.
Accuracy for Alpha-value: 0.9 with fit_prior True is 0.82.
Accuracy for Alpha-value: 1.0 with fit_prior True is 0.813.
Accuracy for Alpha-value: 1.1 with fit_prior True is 0.82.
Accuracy for Alpha-value: 1.2 with fit_prior True is 0.817.
Accuracy for Alpha-value: 1.3 with fit_prior True is 0.813.
Accuracy for Alpha-value: 1.4 with fit_prior True is 0.82.
Accuracy for Alpha-value: 1.5 with fit_prior True is 0.82.
Accuracy for Alpha-value: 1.6 with fit_prior True is 0.82.
Accuracy for Alpha-value: 1.7 with fit_prior True 

### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both accuracy and macro-averaged F-score for each classifier. Be sure to label your output.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using optimal hyper-parameter settings.

In [17]:
from sklearn.metrics import f1_score

# Optimal alpha = 0.2 & fit_prior = false...
clf_multinomial = MultinomialNB(alpha = 0.2, fit_prior = False)
clf_multinomial.fit(X_train_vect, y_train)
y_pred = clf_multinomial.predict(X_test_vect)
print("Multinomial Naive Bayes:")
print("Accuracy for Alpha-value 0.1 with fit_prior False is " + str(round(accuracy_score(y_dev, y_pred), 3)) + ".")
print("F-score: " + str(round(f1_score(y_test, y_pred, average='macro'), 3)))


# Optimum solver is liblinear (which is expected given the size of our dataset) & C = 10...
clf_regression = LogisticRegression(random_state = 0, solver = 'liblinear', C = 10)
clf_regression.fit(X_train_vect, y_train)
y_pred = clf_regression.predict(X_test_vect)
print("\n\nLinear Logisitic Regression:")
print("Accuracy for C-value 10  with solver type liblinear is "+ str(round(accuracy_score(y_dev, y_pred), 3)) + ".")
print("F-score: " + str(round(f1_score(y_test, y_pred, average='macro'), 3)))

Multinomial Naive Bayes:
Accuracy for Alpha-value 0.1 with fit_prior False is 0.823.
F-score: 0.804


Linear Logisitic Regression:
Accuracy for C-value 10  with solver type liblinear is 0.847.
F-score: 0.815
